In [5]:

import datetime

import netCDF4
import pandas as pd
import xarray as xr
import matplotlib as mpl
import cdsapi
import numpy as np
import seaborn as sns
import warnings
import geopandas as gp
import regionmask
import matplotlib.pyplot as plt
import pandas as pd
warnings.filterwarnings('ignore')
from tabulate import tabulate
plt.style.use('default')
import cftime
import cartopy.crs as ccrs
import pvlib
import os
from global_land_mask import globe
import CMIP6_light_map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box, mapping
import geopandas as gpd

def convert_to_180(ds):
    return ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

# Here, we use the ERA5 data and the Python API interface to cdsapi.
# The script downloads ERA5 used to compare with the calculated values of the RTM model.
# The difference between variables is used to bias-correct the RTM model. To use this script you need to install the cdsapi library
# and to have a valid account and credentials. The data is downloaded in netCDF format and saved in the specified path.
#
# https://www.ecmwf.int/sites/default/files/elibrary/2015/18490-radiation-quantities-ecmwf-model-and-mars.pdf
# ERA5, Monthly Means of Daily Forecast Accumulations
# https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form
#
# Trond Kristiansen, May 11 2024

def get_global_era5():
    c = cdsapi.Client()

    years = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010',
             '2011','2012','2013','2014','2015','2016','2017','2018','2019']

    c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'product_type': 'monthly_averaged_reanalysis_by_hour_of_day',
        'variable': [
            'clear_sky_direct_solar_radiation_at_surface',
            'downward_uv_radiation_at_the_surface',
            'surface_solar_radiation_downward_clear_sky',
            'surface_solar_radiation_downwards',
        ],
        'year': years,
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            85, -180, 60,
            180,
        ],
        'format': 'netcdf',
    },
        "/mnt/disks/actea-disk-1/light/era5/ERA5_global_hourly_shortwave.nc")

In [6]:
%%time
get_global_era5()

2024-05-11 17:01:57,109 INFO Welcome to the CDS
2024-05-11 17:01:57,110 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-05-11 17:01:57,374 INFO Request is completed
2024-05-11 17:01:57,375 INFO Downloading https://download-0003-clone.copernicus-climate.eu/cache-compute-0003/cache/data3/adaptor.mars.internal-1715445797.1470056-31143-1-e807009a-fe18-42f9-aa27-4c7986da418c.nc to /mnt/disks/actea-disk-1/light/era5/ERA5_global_hourly_shortwave.nc (6.2G)
2024-05-11 18:45:07,648 INFO Download rate 1M/s        


CPU times: user 3min 21s, sys: 53.6 s, total: 4min 15s
Wall time: 1h 43min 12s
